In [1]:
import numpy as np
import matplotlib.pylab as pl
import pandas as pd
from IPython.display import display
from IPython.display import HTML

In [2]:
def rows_eliminate_lu(A: np.array, B: np.array, ii, jj):
    A_copy = A.copy()
    B_copy = B.copy()
    support_row_index = jj
    if A_copy[support_row_index][jj] == 0:
        raise ValueError('support value must be non-zero')
    support_row_multiplier = A_copy[ii][jj] / A_copy[support_row_index][jj]

    for j in range(len(A[support_row_index])):
        A_copy[support_row_index][j] *= support_row_multiplier
    B_copy[support_row_index] *= support_row_multiplier
    for j in range(len(A[support_row_index])):
        A[ii][j] -= A_copy[support_row_index][j]
    B[ii] -= B_copy[support_row_index]
    return (A, B, support_row_multiplier)

In [3]:
def row_divide(A: np.array, B: np.array, ii, jj):
    n = A[ii][jj]
    for j in range(len(A[ii])):
        A[ii][j] /= n
    B[ii] /= n
    return (A, B)

In [4]:
def LU_no_pivoting(A, B):
    A = A.copy()
    B = B.copy()
    C = np.identity(A.shape[0])
    assert len(A.shape) == 2
    assert len(B.shape) == 1

    for i in range(A.shape[1]):
        # always start from the diag element, so as a result it'll yeild low-echelon matrix
        for j in range(i+1, A.shape[0]):
            el = A[j][i]
            if el != 0:
                A, B, mult = rows_eliminate_lu(A, B, j, i)
                C[j][i] = mult
    return (A, B, C) # C is an L matrix

In [23]:
def swap(A, b, P, i, j):
    A_copy = A.copy()
    b_copy = b.copy()
    P_copy = P.copy()
    temp = A_copy[i].copy()
    A_copy[i] = A_copy[j].copy()
    A_copy[j] = temp
    temp_b = b_copy[i]
    b[i] = b[j]
    b[j] = temp_b
    temp_P = P_copy[i].copy()
    P_copy[i] = P_copy[j].copy()
    P_copy[j] = temp_P
    return (A_copy, b_copy, P_copy)


def LU_pivoting(A, b):
    # sorting
    P = np.identity(A.shape[0])
    for i in range(A.shape[1]):
        for j in range(A.shape[0]-1, i, -1):
            if np.abs(A[j][i]) > np.abs(A[i][i]):
                A, b, P = swap(A, b, P, i, j)
                break
    return LU_no_pivoting(A, b), P.astype(int)

LU without pivoting

In [6]:
BIRTHDAY_MONTH = 4
BIRTHDAY_DAY = 26
random_matrix = (np.random.rand(BIRTHDAY_MONTH+BIRTHDAY_DAY,
                                BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10 + 1).astype(int).astype(np.float32)
# for i in  range(0, 15):
#     random_matrix[i][i] = 0
# random_matrix = np.array([
#     [6, 1, -1, 2],
#     [4, 5, -3, 6],
#     [-2, 3, 4, 6],
#     [4, 2, -4, 2]]).astype(np.float32)

right_side_vector_T = (np.random.rand(
    BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10).astype(int).T
# right_side_vector_T = (np.array([5,9,4,2])).T
U, b, L = LU_no_pivoting(random_matrix, right_side_vector_T)

random_matrix_pd = pd.DataFrame(random_matrix)
right_side_vector_T_pd = pd.DataFrame(right_side_vector_T)
U_pd = pd.DataFrame(U)
b_pd = pd.DataFrame(np.array([b]).T)
L_pd = pd.DataFrame(L)
print('---------------A-------------')
display(random_matrix_pd)
print('---------------b-------------')
display(right_side_vector_T_pd)

print('---------------RESULT-------------')
print('---------------L-------------')
display(np.around(L_pd))
print('---------------U-------------')
display(np.around(U_pd))

print('---------------CHECK-------------')
pd.DataFrame(np.around(np.dot(L, U)))

---------------A-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,9.0,10.0,9.0,3.0,2.0,2.0,2.0,2.0,4.0,7.0,...,5.0,9.0,3.0,2.0,8.0,2.0,9.0,6.0,3.0,9.0
1,3.0,10.0,4.0,7.0,5.0,3.0,10.0,5.0,4.0,3.0,...,3.0,10.0,10.0,6.0,2.0,9.0,5.0,6.0,7.0,7.0
2,7.0,6.0,9.0,9.0,9.0,5.0,2.0,1.0,4.0,5.0,...,9.0,2.0,4.0,2.0,7.0,2.0,7.0,4.0,7.0,7.0
3,2.0,9.0,9.0,10.0,6.0,7.0,8.0,10.0,1.0,2.0,...,6.0,7.0,1.0,2.0,4.0,3.0,2.0,7.0,1.0,8.0
4,7.0,8.0,6.0,9.0,10.0,2.0,3.0,4.0,7.0,5.0,...,2.0,7.0,2.0,10.0,3.0,3.0,8.0,4.0,4.0,5.0
5,4.0,7.0,3.0,4.0,6.0,6.0,8.0,7.0,6.0,1.0,...,10.0,3.0,7.0,1.0,10.0,4.0,10.0,9.0,10.0,7.0
6,2.0,5.0,7.0,5.0,5.0,7.0,1.0,3.0,6.0,1.0,...,10.0,2.0,9.0,6.0,6.0,2.0,8.0,3.0,2.0,7.0
7,4.0,2.0,2.0,1.0,7.0,8.0,4.0,4.0,1.0,8.0,...,2.0,7.0,8.0,2.0,1.0,9.0,1.0,1.0,8.0,4.0
8,6.0,3.0,8.0,3.0,10.0,6.0,8.0,4.0,5.0,1.0,...,4.0,4.0,7.0,5.0,3.0,3.0,8.0,4.0,8.0,1.0
9,7.0,3.0,6.0,1.0,3.0,7.0,1.0,8.0,9.0,7.0,...,5.0,1.0,7.0,6.0,10.0,5.0,4.0,6.0,3.0,9.0


---------------b-------------


,0
0,7
1,0
2,4
3,8
4,4
5,3
6,9
7,5
8,3
9,9


---------------RESULT-------------
---------------L-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,-0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,-1.0,-0.0,7.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,2.0,1.0,7.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,-0.0,-1.0,-0.0,14.0,2.0,-7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,-1.0,1.0,0.0,20.0,2.0,-80.0,11.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,-1.0,-0.0,-0.0,3.0,1.0,12.0,-2.0,-0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---------------U-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,9.0,10.0,9.0,3.0,2.0,2.0,2.0,2.0,4.0,7.0,...,5.0,9.0,3.0,2.0,8.0,2.0,9.0,6.0,3.0,9.0
1,0.0,7.0,1.0,6.0,4.0,2.0,9.0,4.0,3.0,1.0,...,1.0,7.0,9.0,5.0,-1.0,8.0,2.0,4.0,6.0,4.0
2,0.0,0.0,2.0,8.0,9.0,4.0,3.0,1.0,2.0,-0.0,...,5.0,-3.0,4.0,2.0,1.0,3.0,1.0,0.0,6.0,1.0
3,0.0,0.0,0.0,-19.0,-22.0,-7.0,-10.0,4.0,-7.0,0.0,...,-11.0,6.0,-20.0,-9.0,1.0,-13.0,-3.0,1.0,-22.0,-1.0
4,0.0,0.0,0.0,0.0,0.0,-1.0,-3.0,5.0,1.0,-0.0,...,-5.0,2.0,-10.0,4.0,-2.0,-5.0,-1.0,-0.0,-8.0,-2.0
5,0.0,0.0,-0.0,-0.0,0.0,14.0,22.0,-25.0,-3.0,-0.0,...,45.0,-16.0,64.0,-33.0,23.0,30.0,9.0,7.0,58.0,16.0
6,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-14.0,3.0,2.0,...,6.0,-0.0,25.0,4.0,-2.0,11.0,4.0,-6.0,10.0,3.0
7,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-102.0,16.0,23.0,...,29.0,11.0,178.0,33.0,-26.0,88.0,13.0,-46.0,73.0,19.0
8,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,77.0,-101.0,...,155.0,-133.0,6.0,-40.0,149.0,-94.0,140.0,77.0,54.0,16.0
9,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,16.0,...,-26.0,20.0,16.0,24.0,-24.0,22.0,-20.0,-13.0,-9.0,0.0


---------------CHECK-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,9.0,10.0,9.0,3.0,2.0,2.0,2.0,2.0,4.0,7.0,...,5.0,9.0,3.0,2.0,8.0,2.0,9.0,6.0,3.0,9.0
1,3.0,10.0,4.0,7.0,5.0,3.0,10.0,5.0,4.0,3.0,...,3.0,10.0,10.0,6.0,2.0,9.0,5.0,6.0,7.0,7.0
2,7.0,6.0,9.0,9.0,9.0,5.0,2.0,1.0,4.0,5.0,...,9.0,2.0,4.0,2.0,7.0,2.0,7.0,4.0,7.0,7.0
3,2.0,9.0,9.0,10.0,6.0,7.0,8.0,10.0,1.0,2.0,...,6.0,7.0,1.0,2.0,4.0,3.0,2.0,7.0,1.0,8.0
4,7.0,8.0,6.0,9.0,10.0,2.0,3.0,4.0,7.0,5.0,...,2.0,7.0,2.0,10.0,3.0,3.0,8.0,4.0,4.0,5.0
5,4.0,7.0,3.0,4.0,6.0,6.0,8.0,7.0,6.0,1.0,...,10.0,3.0,7.0,1.0,10.0,4.0,10.0,9.0,10.0,7.0
6,2.0,5.0,7.0,5.0,5.0,7.0,1.0,3.0,6.0,1.0,...,10.0,2.0,9.0,6.0,6.0,2.0,8.0,3.0,2.0,7.0
7,4.0,2.0,2.0,1.0,7.0,8.0,4.0,4.0,1.0,8.0,...,2.0,7.0,8.0,2.0,1.0,9.0,1.0,1.0,8.0,4.0
8,6.0,3.0,8.0,3.0,10.0,6.0,8.0,4.0,5.0,1.0,...,4.0,4.0,7.0,5.0,3.0,3.0,8.0,4.0,8.0,1.0
9,7.0,3.0,6.0,1.0,3.0,7.0,1.0,8.0,9.0,7.0,...,5.0,1.0,7.0,6.0,10.0,5.0,4.0,6.0,3.0,9.0


LU with pivoting

In [7]:
BIRTHDAY_MONTH = 4
BIRTHDAY_DAY = 26
random_matrix = (np.random.rand(BIRTHDAY_MONTH+BIRTHDAY_DAY,
                                BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10 + 1).astype(int).astype(np.float32)
for i in  range(0, 15):
    random_matrix[i][i] = 0
# random_matrix = np.array([
#     [6, 1, -1, 2],
#     [4, 5, -3, 6],
#     [-2, 3, 4, 6],
#     [4, 2, -4, 2]]).astype(np.float32)

right_side_vector_T = (np.random.rand(
    BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10).astype(int).T
# right_side_vector_T = (np.array([5,9,4,2])).T
(U, b, L), P = LU_pivoting(random_matrix, right_side_vector_T)

random_matrix_pd = pd.DataFrame(random_matrix)
right_side_vector_T_pd = pd.DataFrame(right_side_vector_T)
U_pd = pd.DataFrame(U)
b_pd = pd.DataFrame(np.array([b]).T)
L_pd = pd.DataFrame(L)
print('---------------A-------------')
display(random_matrix_pd)
print('---------------b-------------')
display(right_side_vector_T_pd)

print('---------------RESULT-------------')
print('---------------L-------------')
display(np.around(L_pd))
print('---------------U-------------')
display(np.around(U_pd))

print('---------------CHECK-------------')
pd.DataFrame(np.around(np.linalg.inv(P)@L@U))

_____TEST______


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3.0,5.0,10.0,4.0,5.0,6.0,8.0,3.0,10.0,10.0,...,7.0,3.0,8.0,1.0,8.0,3.0,8.0,5.0,6.0,2.0
1,0.0,9.0,7.0,2.0,4.0,10.0,5.0,7.0,6.0,1.0,...,4.0,9.0,6.0,1.0,7.0,1.0,8.0,5.0,7.0,7.0
2,9.0,0.0,2.0,10.0,6.0,3.0,6.0,8.0,4.0,1.0,...,7.0,7.0,5.0,3.0,6.0,1.0,1.0,2.0,2.0,4.0
3,1.0,4.0,0.0,9.0,4.0,3.0,8.0,3.0,2.0,5.0,...,7.0,3.0,9.0,10.0,1.0,2.0,7.0,5.0,6.0,8.0
4,9.0,7.0,10.0,0.0,10.0,10.0,4.0,3.0,4.0,9.0,...,5.0,3.0,9.0,4.0,1.0,9.0,3.0,10.0,6.0,9.0
5,5.0,9.0,6.0,9.0,0.0,6.0,10.0,8.0,4.0,8.0,...,4.0,7.0,3.0,3.0,9.0,4.0,5.0,3.0,4.0,10.0
6,9.0,10.0,2.0,10.0,8.0,0.0,4.0,5.0,10.0,2.0,...,1.0,7.0,1.0,7.0,3.0,7.0,5.0,4.0,8.0,7.0
7,8.0,10.0,8.0,8.0,6.0,5.0,0.0,10.0,4.0,4.0,...,6.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,5.0,8.0
8,1.0,2.0,7.0,8.0,5.0,1.0,2.0,0.0,8.0,1.0,...,10.0,1.0,7.0,8.0,1.0,10.0,6.0,1.0,7.0,7.0
9,9.0,2.0,5.0,2.0,3.0,5.0,2.0,5.0,0.0,1.0,...,4.0,10.0,7.0,6.0,3.0,6.0,10.0,5.0,1.0,6.0


---------------A-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,9.0,7.0,2.0,4.0,10.0,5.0,7.0,6.0,1.0,...,4.0,9.0,6.0,1.0,7.0,1.0,8.0,5.0,7.0,7.0
1,9.0,0.0,2.0,10.0,6.0,3.0,6.0,8.0,4.0,1.0,...,7.0,7.0,5.0,3.0,6.0,1.0,1.0,2.0,2.0,4.0
2,1.0,4.0,0.0,9.0,4.0,3.0,8.0,3.0,2.0,5.0,...,7.0,3.0,9.0,10.0,1.0,2.0,7.0,5.0,6.0,8.0
3,9.0,7.0,10.0,0.0,10.0,10.0,4.0,3.0,4.0,9.0,...,5.0,3.0,9.0,4.0,1.0,9.0,3.0,10.0,6.0,9.0
4,5.0,9.0,6.0,9.0,0.0,6.0,10.0,8.0,4.0,8.0,...,4.0,7.0,3.0,3.0,9.0,4.0,5.0,3.0,4.0,10.0
5,9.0,10.0,2.0,10.0,8.0,0.0,4.0,5.0,10.0,2.0,...,1.0,7.0,1.0,7.0,3.0,7.0,5.0,4.0,8.0,7.0
6,8.0,10.0,8.0,8.0,6.0,5.0,0.0,10.0,4.0,4.0,...,6.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,5.0,8.0
7,1.0,2.0,7.0,8.0,5.0,1.0,2.0,0.0,8.0,1.0,...,10.0,1.0,7.0,8.0,1.0,10.0,6.0,1.0,7.0,7.0
8,9.0,2.0,5.0,2.0,3.0,5.0,2.0,5.0,0.0,1.0,...,4.0,10.0,7.0,6.0,3.0,6.0,10.0,5.0,1.0,6.0
9,9.0,9.0,7.0,6.0,9.0,2.0,1.0,7.0,5.0,0.0,...,6.0,8.0,5.0,7.0,9.0,5.0,6.0,8.0,7.0,5.0


---------------b-------------


...

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,-2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,-1.0,1.0,-2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,0.0,1.0,0.0,-2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3.0,-1.0,1.0,-0.0,-0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.0,-0.0,1.0,-0.0,-1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,-0.0,1.0,0.0,-0.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3.0,-1.0,1.0,-1.0,-0.0,0.0,-0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---------------U-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3.0,5.0,10.0,4.0,5.0,6.0,8.0,3.0,10.0,10.0,...,7.0,3.0,8.0,1.0,8.0,3.0,8.0,5.0,6.0,2.0
1,0.0,9.0,7.0,2.0,4.0,10.0,5.0,7.0,6.0,1.0,...,4.0,9.0,6.0,1.0,7.0,1.0,8.0,5.0,7.0,7.0
2,0.0,0.0,-16.0,1.0,-2.0,2.0,-10.0,11.0,-16.0,-27.0,...,-7.0,13.0,-9.0,2.0,-6.0,-6.0,-10.0,-5.0,-4.0,10.0
3,0.0,0.0,0.0,7.0,2.0,-2.0,7.0,-3.0,2.0,10.0,...,6.0,-4.0,8.0,9.0,-1.0,3.0,5.0,4.0,4.0,2.0
4,0.0,0.0,0.0,0.0,4.0,-4.0,5.0,-14.0,-4.0,20.0,...,4.0,-16.0,11.0,15.0,-14.0,11.0,3.0,9.0,4.0,5.0
5,0.0,0.0,0.0,0.0,0.0,-13.0,10.0,-31.0,-9.0,45.0,...,3.0,-38.0,14.0,27.0,-26.0,23.0,3.0,14.0,3.0,9.0
6,0.0,0.0,0.0,0.0,0.0,0.0,-12.0,19.0,19.0,-37.0,...,-8.0,27.0,-19.0,-26.0,23.0,-18.0,-1.0,-16.0,-2.0,-19.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,-18.0,20.0,...,7.0,-21.0,10.0,14.0,-20.0,6.0,-2.0,6.0,-1.0,13.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,-16.0,...,-5.0,15.0,-6.0,-1.0,10.0,11.0,-1.0,-2.0,3.0,2.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,2.0,5.0,7.0,7.0,-1.0,-0.0,14.0,3.0,2.0,-0.0


---------------CHECK-------------


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,9.0,7.0,2.0,4.0,10.0,5.0,7.0,6.0,1.0,...,4.0,9.0,6.0,1.0,7.0,1.0,8.0,5.0,7.0,7.0
1,9.0,0.0,2.0,10.0,6.0,3.0,6.0,8.0,4.0,1.0,...,7.0,7.0,5.0,3.0,6.0,1.0,1.0,2.0,2.0,4.0
2,1.0,4.0,-0.0,9.0,4.0,3.0,8.0,3.0,2.0,5.0,...,7.0,3.0,9.0,10.0,1.0,2.0,7.0,5.0,6.0,8.0
3,9.0,7.0,10.0,-0.0,10.0,10.0,4.0,3.0,4.0,9.0,...,5.0,3.0,9.0,4.0,1.0,9.0,3.0,10.0,6.0,9.0
4,5.0,9.0,6.0,9.0,0.0,6.0,10.0,8.0,4.0,8.0,...,4.0,7.0,3.0,3.0,9.0,4.0,5.0,3.0,4.0,10.0
5,9.0,10.0,2.0,10.0,8.0,-0.0,4.0,5.0,10.0,2.0,...,1.0,7.0,1.0,7.0,3.0,7.0,5.0,4.0,8.0,7.0
6,8.0,10.0,8.0,8.0,6.0,5.0,0.0,10.0,4.0,4.0,...,6.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,5.0,8.0
7,1.0,2.0,7.0,8.0,5.0,1.0,2.0,0.0,8.0,1.0,...,10.0,1.0,7.0,8.0,1.0,10.0,6.0,1.0,7.0,7.0
8,9.0,2.0,5.0,2.0,3.0,5.0,2.0,5.0,-0.0,1.0,...,4.0,10.0,7.0,6.0,3.0,6.0,10.0,5.0,1.0,6.0
9,9.0,9.0,7.0,6.0,9.0,2.0,1.0,7.0,5.0,-0.0,...,6.0,8.0,5.0,7.0,9.0,5.0,6.0,8.0,7.0,5.0


In [24]:
A_test = np.array([
    [0,    0.6324,    0.9575,    0.9572],
    [0.9058,    0,    0.9649,    0.4854],
    [0.1270,    0.2785,    0,    0.8003],
    [0.9134,    0.5469,    0.9706,    0],
])
(U, b, L), P = LU_pivoting(A_test, np.array([1, 2, 3, 4]).T)

In [25]:
P_inv = np.linalg.inv(P)
P_inv @ L @ U

array([[0.    , 0.6324, 0.9575, 0.9572],
       [0.9058, 0.    , 0.9649, 0.4854],
       [0.127 , 0.2785, 0.    , 0.8003],
       [0.9134, 0.5469, 0.9706, 0.    ]])

In [28]:
P

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [ ]:
0.9134, 0.5469, 0.9706, 0.1419
0.9058, 0.0975, 0.9649, 0.4854
0.8147, 0.6324, 0.9575, 0.9572
0.127 , 0.2785, 0.1576, 0.8003